# 1. Импорт модулей.

In [ ]:
from google.colab import drive
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.models import Sequential, Model
import numpy as np
import pandas as pd
import os
import tensorflow.keras.backend as K
from sklearn.metrics import *
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.preprocessing import MinMaxScaler
import pickle
import seaborn as sns
from matplotlib import pyplot as plt 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# 2. Разделение классов аттак

Packet Length Variance      
Fwd IAT Std                 
Active Min                  
Active Mean                  
Active Max                    
Flow IAT Std                
Bwd IAT Mean                
PSH Flag Count                 
min_seg_size_forward         
Flow IAT Mean               
Active Std                  
Bwd IAT Min                  
Bwd IAT Max                   
Total Length of Fwd Packets    
Subflow Fwd Bytes            
Fwd PSH Flags                 
SYN Flag Count                
Fwd IAT Mean               
Bwd IAT Std                
Flow IAT Min                
Bwd Packet Length Min        
Init_Win_bytes_forward        
Fwd IAT Min        
Label

In [ ]:
file = pd.read_csv('/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/x_smt.csv')
file.info()

In [ ]:
data = []
target = []
for n, i in file.iterrows():
  a = []
  for j in i[:-1]:
    a.append(j)
  data.append(a)
  target.append(0 if i[-1] == 'BENIGN' else 1) #0 benign 1 attack
del file

In [ ]:
print('data example:')
print(data[:2])
print('target example:')
print(target[:2])

data example:
[[8424.2568359375, 50911, 5268490.0, 63678.19921875, 103175, 32, 4901981.5, 7518953.5, 1, 3541331.75, 322, 322, 637, 22252.53515625, 10300000, 0, 0, 6396442.0, 4645137.5, 47, 29200, 0, 234], [0.0, 0, 0.0, 0.0, 0, 32, 0.0, 0.0, 0, 57891.0, 0, 0, 0, 0.0, 0, 0, 0, 0.0, 0.0, 57891, 237, 0, 0]]
target example:
[1, 1]


Создание двух csv из target_df как набора 1 и 0 из верных маркировок и data_df как набора из признаков.

In [ ]:
#создание csv из таргет и даты
#target_df = pd.DataFrame(target)
#csv_target_df = target_df.to_csv('/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/smt_y_bin.csv')
#data_df = pd.DataFrame(data)
#csv_data_df = data_df.to_csv('/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/smt_x_bin.csv')

In [ ]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(np.nan_to_num(data).astype(float))
data = scaler.transform(np.nan_to_num(data).astype(float))
print(data.shape)
#data = data.reshape(data.shape[0], 23)

(283771, 23)


# 3. Решейпинг

In [ ]:
data = np.array(data)
target = np.array(target)
print(data.shape, target.shape)
print('----')
print(data[0].shape)
print('----')
print(data[:2])
print('----')


(283771, 23) (283771,)
----
(23,)
----
[[-0.99924106 -0.99881464 -0.8720885  -0.99851739 -0.99791566 -0.91655802
  -0.88029349 -0.87468411  1.         -0.93601997 -0.99977531 -0.99977531
  -0.99998938 -0.99936421 -0.82833333 -1.         -1.         -0.89339263
  -0.88656563 -0.99999892 -0.1088562  -1.         -0.9999961 ]
 [-1.         -1.         -1.         -1.         -1.         -0.91655802
  -1.         -1.         -1.         -0.99895387 -1.         -1.
  -1.         -1.         -1.         -1.         -1.         -1.
  -1.         -0.99895387 -0.99273682 -1.         -1.        ]]
----


In [ ]:
data = data.reshape(data.shape[0], data.shape[1], 1)
print(f"shape of data", data.shape)

shape of data (283771, 23, 1)


# 4. Моделирование бинарной нейросети LSTM.

In [ ]:
tf.keras.backend.clear_session()

#n_features = data_reshaped.shape[2]

lstm = Sequential()
lstm.add(Input((23, 1))) #lstm.add(Input((1, 23))) None - любое положительное целое число 
lstm.add(LSTM(32, recurrent_activation='sigmoid', return_sequences=True, recurrent_dropout=0))
for i in range(4):
  lstm.add(LSTM(32, recurrent_activation='sigmoid', return_sequences=True, recurrent_dropout=0))
lstm.add(Flatten())
lstm.add(Dense(2 ,activation = 'softmax'))
lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 23, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 23, 32)            8320      
                                                                 
 lstm_2 (LSTM)               (None, 23, 32)            8320      
                                                                 
 lstm_3 (LSTM)               (None, 23, 32)            8320      
                                                                 
 lstm_4 (LSTM)               (None, 23, 32)            8320      
                                                                 
 flatten (Flatten)           (None, 736)               0         
                                                                 
 dense (Dense)               (None, 2)                 1

Тренировка нейросети

In [ ]:
lstm.fit(data, to_categorical(target), epochs=50, batch_size=256, verbose=1)

Epoch 1/50
1109/1109 [==============================] - 143s 118ms/step - loss: 0.5596 - accuracy: 0.7275
Epoch 2/50
1109/1109 [==============================] - 133s 120ms/step - loss: 0.5170 - accuracy: 0.7577
Epoch 3/50
1109/1109 [==============================] - 130s 117ms/step - loss: 0.5152 - accuracy: 0.7568
Epoch 4/50
1109/1109 [==============================] - 145s 131ms/step - loss: 0.4948 - accuracy: 0.7672
Epoch 5/50
1109/1109 [==============================] - 135s 121ms/step - loss: 0.4850 - accuracy: 0.7718
Epoch 6/50
1109/1109 [==============================] - 128s 115ms/step - loss: 0.4910 - accuracy: 0.7660
Epoch 7/50
1109/1109 [==============================] - 127s 115ms/step - loss: 0.5088 - accuracy: 0.7474
Epoch 8/50
1109/1109 [==============================] - 128s 115ms/step - loss: 0.4893 - accuracy: 0.7665
Epoch 9/50
1109/1109 [==============================] - 126s 113ms/step - loss: 0.4363 - accuracy: 0.8027
Epoch 10/50
1109/1109 [=======================

In [ ]:
pickle.dump(lstm, open('/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/lstm.pkl', 'wb'))

Получение данных от нейросети

In [ ]:
y_pred1 = lstm.predict(data, batch_size=256)
y_pred = np.argmax(y_pred1, axis=1)

Вычисление метрик 

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
target_names = ['Benign', 'Attack']
print('==============================Classification Report=============================='.format(i))
print(classification_report(target, y_pred, target_names=target_names))
print('==============================Confussion Matrix and metrics=============================='.format(i))
tn, fp, fn, tp = confusion_matrix(target, y_pred).ravel()
sum = tn + fp + fn + tp
print('Confussion Matrix:', "\n", confusion_matrix(target, y_pred))
print("Accuracy: ", accuracy_score(target, y_pred))
recall = tp / (tp + fn)
print('recall:', recall)
precision = tp / (tp + fp)
print('precision:', precision)
print('f1 score:', 2 * (precision * recall) / (precision + recall))
print('ROC AUC:', roc_auc_score(target, y_pred))
unique, counts = np.unique(y_pred, return_counts=True)
count_list = dict(zip(unique, counts))
print('Alghorythm marked', count_list[1], 'as attack and', count_list[0], "packets as benign")

==============================Classification Report==============================
              precision    recall  f1-score   support

      Benign       0.98      0.95      0.96    142069
      Attack       0.95      0.99      0.97    141702

    accuracy                           0.97    283771
   macro avg       0.97      0.97      0.97    283771
weighted avg       0.97      0.97      0.97    283771

==============================Confussion Matrix and metrics==============================
Confussion Matrix: 
 [[134362   7707]
 [  2110 139592]]
Accuracy:  0.9654052034915478
recall: 0.9851095961948314
precision: 0.9476778525312459
f1 score: 0.9660312594074068
ROC AUC: 0.9654306541955089
Alghorythm marked 147299 as attack and 136472 packets as benign
